In [15]:
import psycopg2
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sqlalchemy import create_engine
from shapely.geometry import Point, LineString
from shapely import wkt

from config import DB_CONFIG


In [16]:
conn = psycopg2.connect(**DB_CONFIG)

if conn.closed == 0:
    print("Connected to DB")

all_points_sql = 'SELECT * FROM segmented_trajectories ORDER BY mmsi ASC,points DESC LIMIT 100000'

seg_traj =  gpd.GeoDataFrame.from_postgis(all_points_sql, conn, geom_col='geometry')


Connected to DB


In [17]:
seg_traj.head(500)

,mmsi,starting_on,ending_on,points,geometry
0,642167061,2018-01-01 00:54:38,2018-01-15 20:35:04,39,"LINESTRING (23.61625 37.81482, 23.65744 37.837..."
1,642167061,2018-01-01 00:24:18,2018-01-01 00:50:17,14,"LINESTRING (23.48296 37.83409, 23.48442 37.833..."
2,667005041,2018-01-09 06:21:35,2018-01-09 07:23:45,250,"LINESTRING (23.66069 37.84075, 23.65996 37.842..."
3,667005041,2018-01-09 07:24:55,2018-01-09 07:50:55,131,"LINESTRING (23.61366 37.90261, 23.61367 37.902..."
4,667005041,2018-01-09 08:14:25,2018-01-09 08:35:55,102,"LINESTRING (23.60248 37.93727, 23.60249 37.938..."
...,...,...,...,...,...
224,677083400,2018-01-24 17:02:38,2018-01-26 16:38:05,11,"LINESTRING (23.55005 37.95493, 23.55005 37.954..."
225,677083400,2018-01-26 16:44:06,2018-01-26 16:53:13,5,"LINESTRING (23.59756 37.92680, 23.59755 37.926..."
226,677083400,2018-01-22 20:53:15,2018-01-24 13:30:26,2,"LINESTRING (23.55055 37.95504, 23.55051 37.95517)"
227,677083400,2018-01-26 19:52:52,2018-01-26 19:53:03,2,"LINESTRING (23.61460 37.92277, 23.61458 37.92270)"


In [18]:
seg_traj=seg_traj.set_crs(4326)
seg_traj = seg_traj.to_crs(2100)

In [19]:
seg_traj.head()

,mmsi,starting_on,ending_on,points,geometry
0,642167061,2018-01-01 00:54:38,2018-01-15 20:35:04,39,"LINESTRING (466073.445 4185050.944, 469708.471..."
1,642167061,2018-01-01 00:24:18,2018-01-01 00:50:17,14,"LINESTRING (454353.403 4187245.499, 454481.342..."
2,667005041,2018-01-09 06:21:35,2018-01-09 07:23:45,250,"LINESTRING (469995.582 4187912.741, 469931.280..."
3,667005041,2018-01-09 07:24:55,2018-01-09 07:50:55,131,"LINESTRING (465885.928 4194791.864, 465886.377..."
4,667005041,2018-01-09 08:14:25,2018-01-09 08:35:55,102,"LINESTRING (464919.591 4198642.137, 464920.235..."


In [8]:
seg_traj.iloc[1,:]['geometry'].length

10590.942493168586

In [20]:
def traj_distance(traj1,traj2):
    
    ctrDist = abs(traj1['geometry'].centroid.distance(traj2['geometry'].centroid))
    traj1_length = abs(traj1['geometry'].length)
    traj2_length = abs(traj2['geometry'].length)
    
    
    
    s1_first, s1_last = traj1['geometry'].boundary
    
    s1 = s1_first.distance(s1_last)
    
    s2_first, s2_last = traj2['geometry'].boundary
        
    s2 = s2_first.distance(s2_last)
    
    s_matrix = np.array([[s1_first.x,s1_first.y],[s2_first.x,s2_last.y]])
    avg = sum([abs(s1),abs(s2)])/2
    
    
    dot_product = np.dot(s_matrix.T[0,:],s_matrix.T[1,:])
    sum_squares = np.sqrt(np.sum(np.square(s_matrix.T[0,:])))*np.sqrt(np.sum(np.square(s_matrix.T[1,:])))
    cos_sim = dot_product/sum_squares
    

    
    distance = ctrDist + ctrDist*abs((traj1_length-traj2_length)/max(traj1_length,traj2_length)) + (-1*avg*cos_sim)
    
    return distance

In [10]:
traj_distance(seg_traj.iloc[0,:],seg_traj.iloc[1,:])

1469.8568997369493

In [11]:
#Trajecotries with similary geographical trajectories

#Similairty calculated using Geographic Distance from the paper:
#https://www.cise.ufl.edu/~mschneid/Research/papers/LS12IWGS.pdf?fbclid=IwAR1vcSg4sRxdz7qBaSr9vmRiu3lMe0WryzRlZzPzheUcE9fH6RexVuZzSBQ
#
# With the similairty distance formula being:
    
#     geoDist(tra1, tra2) = ctrDist(tra1, tra2)
#     + ctrDist(tra1, tra2) × | ‖tra1‖−‖tra2‖ |
#     max(‖tra1, ‖tra2‖)−avg(‖s1‖, ‖s2‖) ×cos(s1, s2) 



similar_traj = []
leader_traj = seg_traj.iloc[2,:]
for index,row in seg_traj.iterrows():
    if traj_distance(row,leader_traj) < 500:
        similar_traj.append(row)

In [12]:
len(similar_traj)

4

In [16]:
gdf = gpd.GeoDataFrame(similar_traj, columns=('mmsi','starting_on','ending_on','points' ,'geometry'))


In [17]:
gdf.head()


,mmsi,starting_on,ending_on,points,geometry
0,642167061,2018-01-01 00:54:38,2018-01-15 20:35:04,39,"LINESTRING (466073.445 4185050.944, 469708.471..."
2,667005041,2018-01-09 06:21:35,2018-01-09 07:23:45,250,"LINESTRING (469995.582 4187912.741, 469931.280..."
83,671389000,2018-01-10 16:19:36,2018-01-10 17:31:33,305,"LINESTRING (469791.470 4186956.343, 469322.137..."
84,671389000,2018-01-11 18:49:25,2018-01-11 20:04:34,235,"LINESTRING (461040.103 4195435.081, 461033.081..."


In [228]:
engine = create_engine(f'postgresql://{DB_CONFIG["user"]}:{DB_CONFIG["password"]}@{DB_CONFIG["host"]}:5433/{DB_CONFIG["database"]}')

In [229]:
gdf.to_postgis('similar_trajectories', engine)

In [22]:
#Trajecotries with similar geographical trajectories, in a given time period(e.g Date)

similar_day_traj = []
leader_traj = seg_traj.iloc[2,:]
for index,row in seg_traj.iterrows():
    if traj_distance(row,leader_traj) < 500:
        if (row['starting_on']-leader_traj['starting_on']).days <= 0:
            similar_day_traj.append(row)
            


In [23]:
similar_day_traj

[mmsi                                                   642167061
 starting_on                                  2018-01-01 00:54:38
 ending_on                                    2018-01-15 20:35:04
 points                                                        39
 geometry       LINESTRING (466073.4447602328 4185050.94364822...
 Name: 0, dtype: object,
 mmsi                                                   667005041
 starting_on                                  2018-01-09 06:21:35
 ending_on                                    2018-01-09 07:23:45
 points                                                       250
 geometry       LINESTRING (469995.582160474 4187912.741008276...
 Name: 2, dtype: object]